In [1]:
import parmed as pmd
import subprocess as sp
import numpy as np

from openeye.oechem import *
from openeye.oeomega import *
from openforcefield.typing.engines.smirnoff import *
from pdbfixer import PDBFixer

In [6]:
ben_smiles = '[O-]C(=O)c1ccccc1'
bcd_smiles = 'C([C@@H]1[C@@H]2[C@@H]([C@H]([C@H](O1)O[C@@H]3[C@H](O[C@@H]([C@@H]([C@H]3O)O)O[C@@H]4[C@H](O[C@@H]([C@@H]([C@H]4O)O)O[C@@H]5[C@H](O[C@@H]([C@@H]([C@H]5O)O)O[C@@H]6[C@H](O[C@@H]([C@@H]([C@H]6O)O)O[C@@H]7[C@H](O[C@@H]([C@@H]([C@H]7O)O)O[C@@H]8[C@H](O[C@H](O2)[C@@H]([C@H]8O)O)CO)CO)CO)CO)CO)CO)O)O)O'

In [7]:
mols = []
for smiles in [ben_smiles, bcd_smiles]:
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, smiles)
    for atom in mol.GetAtoms():
        atom.SetPartialCharge(atom.GetFormalCharge())
    oechem.OEAddExplicitHydrogens(mol)
    oechem.OETriposAtomNames(mol)
    mols.append(mol)

In [8]:
ofs = oemolostream()
ofs.open('bcd-ben/ben-openeye-no-charge.mol2')
OEWriteMolecule(ofs, mols[0])

ofs = oemolostream()
ofs.open('bcd-ben/bcd-openeye-no-charge.mol2')
OEWriteMolecule(ofs, mols[1])

0

Now, start with a blank molecules list.

In [12]:
mols = []

In [13]:
ifs = oechem.oemolistream('bcd-ben/ben-openeye-no-charge.mol2')
for mol in ifs.GetOEGraphMols():
    mols.append(oechem.OEMol(mol))
ifs = oechem.oemolistream('bcd-ben/bcd-openeye-no-charge.mol2')
for mol in ifs.GetOEGraphMols():
    mols.append(oechem.OEMol(mol))

In [14]:
smiles = ['[Na+]', '[Cl-]', 'O']
for molecule in smiles:
    mol = oechem.OEMol()
    oechem.OESmilesToMol(mol, molecule)
    for atom in mol.GetAtoms():
        atom.SetPartialCharge(atom.GetFormalCharge())
    oechem.OEAddExplicitHydrogens(mol)
    oechem.OETriposAtomNames(mol)
    mols.append(mol)

In [15]:
mols

[<openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7faa84c4e7b0> >,
 <openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7faa8af75570> >,
 <openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7faa3cb048a0> >,
 <openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7faa3cb04420> >,
 <openeye.oechem.OEMol; proxy of <Swig Object of type 'OEMolWrapper *' at 0x7faa3cd23f60> >]

In [16]:
pdb = PDBFixer('bcd-ben/bcd-ben-conect.pdb')

In [17]:
ff = ForceField('forcefield/smirnoff99Frosst.ffxml', 'forcefield/tip3p.ffxml') 
system = ff.createSystem(pdb.topology, 
                         mols)
integrator = openmm.VerletIntegrator(1.0 * unit.femtoseconds)
context = openmm.Context(system, integrator)
context.setPositions(pdb.positions)

In [18]:
structure = pmd.openmm.topsystem.load_topology(pdb.topology, system, pdb.positions)

In [20]:
structure.save('bcd-ben/' + 'bcd-ben-build-from-smiles-but-imported-as-mol2.prmtop')

AttributeError: 'NoneType' object has no attribute 'used'